In [1]:
import os
import pandas as pd
import numpy as np
import dataframe as df
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchmetrics
from torchmetrics.functional import accuracy

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
X_train = pd.read_csv('../data2.csv')

In [4]:
X_train.head()

,time,id,group,state,HEOL,HEOR,FP1,FP2,VEOU,VEOL,...,P4,T6,A2,O1,OZ,O2,FT9,FT10,PO1,PO2
0,0,0,0,Fatigue,1.215492e-13,-4.869703e-17,-6.140989e-14,2.598510e-17,-1.160435e-13,-3.167843e-17,...,1.651714e-13,-3.307430e-17,3.455894e-13,-6.391263e-17,8.724439e-14,-3.155565e-17,-7.326835e-14,-3.155565e-17,4.404571e-14,-2.665396e-17
1,1,1,0,Fatigue,-7.831237e+01,-1.362717e-02,-5.690807e+01,-4.192949e-02,3.212726e+02,-1.397650e-02,...,-2.646920e+02,-6.988248e-03,-2.281396e+02,4.192949e-02,-5.818660e+01,3.494124e-02,1.040445e+02,3.494124e-02,-3.291532e+01,2.795299e-02
2,2,2,0,Fatigue,-1.533161e+02,-2.671759e-02,-1.114424e+02,-8.209756e-02,6.290321e+02,-2.736585e-02,...,-5.183021e+02,-1.368293e-02,-4.459774e+02,8.209756e-02,-1.138533e+02,6.841464e-02,2.037677e+02,6.841464e-02,-6.439023e+01,5.473171e-02
3,3,3,0,Fatigue,-2.219502e+02,-3.877185e-02,-1.614074e+02,-1.188756e-01,9.107824e+02,-3.962519e-02,...,-7.505852e+02,-1.981260e-02,-6.440388e+02,1.188756e-01,-1.646722e+02,9.906298e-02,2.951723e+02,9.906298e-02,-9.309419e+01,7.925038e-02
4,4,4,0,Fatigue,-2.816333e+02,-4.936209e-02,-2.049461e+02,-1.508884e-01,1.155975e+03,-5.029612e-02,...,-9.528851e+02,-2.514806e-02,-8.144362e+02,1.508884e-01,-2.086840e+02,1.257403e-01,3.748789e+02,1.257403e-01,-1.179092e+02,1.005922e-01


In [5]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(X_train.state)

label_encoder.classes_

array(['Fatigue', 'Normal'], dtype=object)

In [6]:
X_train['label'] = encoded_labels
X_train.head()

,time,id,group,state,HEOL,HEOR,FP1,FP2,VEOU,VEOL,...,T6,A2,O1,OZ,O2,FT9,FT10,PO1,PO2,label
0,0,0,0,Fatigue,1.215492e-13,-4.869703e-17,-6.140989e-14,2.598510e-17,-1.160435e-13,-3.167843e-17,...,-3.307430e-17,3.455894e-13,-6.391263e-17,8.724439e-14,-3.155565e-17,-7.326835e-14,-3.155565e-17,4.404571e-14,-2.665396e-17,0
1,1,1,0,Fatigue,-7.831237e+01,-1.362717e-02,-5.690807e+01,-4.192949e-02,3.212726e+02,-1.397650e-02,...,-6.988248e-03,-2.281396e+02,4.192949e-02,-5.818660e+01,3.494124e-02,1.040445e+02,3.494124e-02,-3.291532e+01,2.795299e-02,0
2,2,2,0,Fatigue,-1.533161e+02,-2.671759e-02,-1.114424e+02,-8.209756e-02,6.290321e+02,-2.736585e-02,...,-1.368293e-02,-4.459774e+02,8.209756e-02,-1.138533e+02,6.841464e-02,2.037677e+02,6.841464e-02,-6.439023e+01,5.473171e-02,0
3,3,3,0,Fatigue,-2.219502e+02,-3.877185e-02,-1.614074e+02,-1.188756e-01,9.107824e+02,-3.962519e-02,...,-1.981260e-02,-6.440388e+02,1.188756e-01,-1.646722e+02,9.906298e-02,2.951723e+02,9.906298e-02,-9.309419e+01,7.925038e-02,0
4,4,4,0,Fatigue,-2.816333e+02,-4.936209e-02,-2.049461e+02,-1.508884e-01,1.155975e+03,-5.029612e-02,...,-2.514806e-02,-8.144362e+02,1.508884e-01,-2.086840e+02,1.257403e-01,3.748789e+02,1.257403e-01,-1.179092e+02,1.005922e-01,0


In [7]:
FEATURE_COLUMNS = X_train.columns.tolist()[4:-2]

In [8]:
g = X_train.groupby("group")

In [9]:
sequences = [] 

for name, group in g:
    sequence_features = group[FEATURE_COLUMNS]
    label = group.label.iloc[0]
    
#     print((sequence_features, label))
    sequences.append((sequence_features, label))

In [10]:
sequences[0]

(              HEOL          HEOR           FP1           FP2          VEOU  \
 0     1.215492e-13 -4.869703e-17 -6.140989e-14  2.598510e-17 -1.160435e-13   
 1    -7.831237e+01 -1.362717e-02 -5.690807e+01 -4.192949e-02  3.212726e+02   
 2    -1.533161e+02 -2.671759e-02 -1.114424e+02 -8.209756e-02  6.290321e+02   
 3    -2.219502e+02 -3.877185e-02 -1.614074e+02 -1.188756e-01  9.107824e+02   
 4    -2.816333e+02 -4.936209e-02 -2.049461e+02 -1.508884e-01  1.155975e+03   
 ...            ...           ...           ...           ...           ...   
 1019  9.189533e+00  2.849707e-03  1.783298e+01  6.329993e-03 -4.930598e+01   
 1020  7.436606e+00  2.869100e-03  1.661349e+01  6.348566e-03 -5.123579e+01   
 1021  5.528420e+00  2.921818e-03  1.508811e+01  6.324462e-03 -5.286875e+01   
 1022  3.726376e+00  3.024905e-03  1.349644e+01  6.342777e-03 -5.465339e+01   
 1023  1.962049e+00  3.137497e-03  1.184765e+01  6.318416e-03 -5.574251e+01   
 
               VEOL            F7            F3   

In [11]:
train_sequences, test_sequences = train_test_split(sequences, test_size=0.2)

In [12]:
len(train_sequences), len(test_sequences)

(11268, 2818)

## Data Set

In [13]:
class DrowsyDataset(Dataset):
    
    def __init__(self, sequences):
        self.sequences = sequences
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        sequence, label = self.sequences[idx]
        return dict(
            sequence=torch.Tensor(sequence.to_numpy()),
            label=torch.tensor(label).long()
        )

In [14]:
class DrowsyDataModule(pl.LightningDataModule):
    
    def __init__(self, train_sequences, test_sequences, batch_size):
        super().__init__()
        self.train_sequences = train_sequences
        self.test_sequences = test_sequences
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train_dataset = DrowsyDataset(self.train_sequences)
        self.test_dataset = DrowsyDataset(self.test_sequences)
        
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=os.cpu_count()
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=os.cpu_count()
        )
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=os.cpu_count()
        ) 

In [15]:
N_EPOCHS = 128
BATCH_SIZE = 100

data_module = DrowsyDataModule(train_sequences, test_sequences, BATCH_SIZE)

## Models

In [16]:
class DrowsyModel(nn.Module):
    
    def __init__(self, n_features, n_classes, n_hidden=256, n_layers=3):
        super().__init__()
        
        self.n_hidden = n_hidden
        
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            num_layers=n_layers,
            batch_first=True,
            dropout=0.75
        )
        
        self.classifier = nn.Linear(n_hidden, n_classes)
        
    def forward(self, x):
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)
        
        out = hidden[-1]
        return self.classifier(out)

In [17]:
class DrowsyPredictor(pl.LightningModule):
    
    def __init__(self, n_features: int, n_classes: int):
        super().__init__()
        self.model = DrowsyModel(n_features, n_classes)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x, label=None):
        output = self.model(x)
        loss = 0
        if label is not None:
            loss = self.criterion(output, label)
        return loss, output
        
    def training_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels)
        
        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.log("train_accuracy", step_accuracy, prog_bar=True, logger=True)
        
        return {"loss": loss, "accuracy": step_accuracy}
    
    def validation_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels)
        
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.log("val_accuracy", step_accuracy, prog_bar=True, logger=True)
        
        return {"loss": loss, "accuracy": step_accuracy}
    
    def testing_step(self, batch, batch_idx):
        sequences = batch["sequences"]
        labels = batch["label"]
        loss, outputs = self(sequence, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels)
        
        self.log("test_loss", loss, prog_bar=True, logger=True)
        self.log("test_accuracy", step_accuracy, prog_bar=True, logger=True)
        
        return {"loss": loss, "accuracy": step_accuracy}
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.0001)

In [18]:
model = DrowsyPredictor(
    n_features=len(FEATURE_COLUMNS), 
    n_classes=len(label_encoder.classes_)
)

In [19]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

logger = TensorBoardLogger("lightning_logs", name="Drowsy")

trainer = pl.Trainer(
    logger=logger,
    callbacks=[checkpoint_callback],
    max_epochs=N_EPOCHS,
    gpus=[0],
#     progress_bar_refresh_rate=20
)

## Training 

In [ ]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DrowsyModel      | 1.4 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.429     Total estimated model params size (MB)


Epoch 0:  80%|███████▉  | 113/142 [00:52<00:13,  2.14it/s, loss=0.672, v_num=0, train_loss=0.652, train_accuracy=0.676]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 142/142 [00:58<00:00,  2.43it/s, loss=0.672, v_num=0, train_loss=0.652, train_accuracy=0.676, val_loss=0.645, val_accuracy=0.621]
                                                           

Epoch 0, global step 112: val_loss reached 0.64530 (best 0.64530), saving model to "/Workspace/code/checkpoints/sample-mnist-epoch=00-val_loss=0.65.ckpt" as top 1


Epoch 1:  80%|████████  | 114/142 [00:53<00:13,  2.13it/s, loss=0.576, v_num=0, train_loss=0.691, train_accuracy=0.574, val_loss=0.645, val_accuracy=0.621]
Validating: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 142/142 [00:59<00:00,  2.38it/s, loss=0.576, v_num=0, train_loss=0.691, train_accuracy=0.574, val_loss=0.547, val_accuracy=0.739]
                                                           

Epoch 1, global step 225: val_loss reached 0.54734 (best 0.54734), saving model to "/Workspace/code/checkpoints/sample-mnist-epoch=01-val_loss=0.55.ckpt" as top 1


...

Epoch 20:  80%|████████  | 114/142 [00:54<00:13,  2.10it/s, loss=0.515, v_num=0, train_loss=0.601, train_accuracy=0.706, val_loss=0.570, val_accuracy=0.719]
Validating: 0it [00:00, ?it/s]
Epoch 20: 100%|██████████| 142/142 [01:00<00:00,  2.33it/s, loss=0.515, v_num=0, train_loss=0.601, train_accuracy=0.706, val_loss=0.543, val_accuracy=0.745]
                                                           

Epoch 20, global step 2372: val_loss was not in top 1


Epoch 21:  80%|████████  | 114/142 [00:54<00:13,  2.10it/s, loss=0.514, v_num=0, train_loss=0.497, train_accuracy=0.721, val_loss=0.543, val_accuracy=0.745]
Validating: 0it [00:00, ?it/s]
Epoch 21: 100%|██████████| 142/142 [01:00<00:00,  2.34it/s, loss=0.514, v_num=0, train_loss=0.497, train_accuracy=0.721, val_loss=0.539, val_accuracy=0.743]
                                                           

Epoch 21, global step 2485: val_loss was not in top 1


Epoch 22:  48%|████▊     | 68/142 [00:33<00:36,  2.01it/s, loss=0.514, v_num=0, train_loss=0.516, train_accuracy=0.770, val_loss=0.539, val_accuracy=0.743] 

In [ ]:
trainer.test()

## Train from check point

In [ ]:
# trainer.fit(model, data_module, ckpt_path="/home/masalu/checkpoints/best-checkpoint.ckpt")